LSTM!!!!!!

In [1]:
import numpy as np
import pandas as pd
import os
import pickle
import re
import warnings
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import torch #pytorch
import torch.nn as nn
from torch.autograd import Variable 

In [2]:
os.getcwd()
# from thon.Choose import choose_features
from thon.churn_functions import modernize, tree_importance, plot_eval, bake
#data = choose_features("data/out/features.csv", cv_range = (0, 30), save = False)

In [3]:
data = pd.read_csv("data/out/features.csv")
X, y = data.drop(columns = ['n']), data[['n']]


In [4]:
mm = MinMaxScaler()
ss = StandardScaler()

X_ss = ss.fit_transform(X)
y_ss = mm.fit_transform(y)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_ss, y_ss, test_size = .1, shuffle = False, )

In [6]:
X_train_tensors = Variable(torch.Tensor(X_train))
X_test_tensors = Variable(torch.Tensor(X_test))

y_train_tensors = Variable(torch.Tensor(y_train))
y_test_tensors = Variable(torch.Tensor(y_test)) 

In [7]:
#reshaping to rows, timestamps, features
# is this the right shape?
X_train_tensors_final = torch.reshape(X_train_tensors,   (X_train_tensors.shape[0], 1, X_train_tensors.shape[1]))


X_test_tensors_final = torch.reshape(X_test_tensors,  (X_test_tensors.shape[0], 1, X_test_tensors.shape[1]))

In [8]:
print("Training Shape", X_train_tensors_final.shape, y_train_tensors.shape)
print("Testing Shape", X_test_tensors_final.shape, y_test_tensors.shape) 

Training Shape torch.Size([135, 1, 74]) torch.Size([135, 1])
Testing Shape torch.Size([15, 1, 74]) torch.Size([15, 1])


In [9]:
class LSTM1(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(LSTM1, self).__init__()
        self.num_classes = num_classes #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True) #lstm
        self.fc_1 =  nn.Linear(hidden_size, 128) #fully connected 1
        self.fc = nn.Linear(128, num_classes) #fully connected last layer

        self.relu = nn.ReLU()
    
    def forward(self,x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) #hidden state
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) #internal state
        # Propagate input through LSTM
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
        hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
        out = self.relu(hn)
        out = self.fc_1(out) #first Dense
        out = self.relu(out) #relu
        out = self.fc(out) #Final Output
        return out

In [10]:
num_epochs = 1000 #1000 epochs
learning_rate = 0.001 #0.001 lr

input_size = 74 #number of features
hidden_size = 2 #number of features in hidden state
num_layers = 1 #number of stacked lstm layers

num_classes = 1 #number of output classes 

In [11]:
lstm1 = LSTM1(num_classes, input_size, hidden_size, num_layers, X_train_tensors_final.shape[1]) #our lstm class 


In [12]:
criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm1.parameters(), lr=learning_rate) 

In [13]:
for epoch in range(num_epochs):
  outputs = lstm1.forward(X_train_tensors_final) #forward pass
  optimizer.zero_grad() #caluclate the gradient, manually setting to 0
 
  # obtain the loss function
  loss = criterion(outputs, y_train_tensors)
 
  loss.backward() #calculates the loss of the loss function
 
  optimizer.step() #improve from loss, i.e backprop
  if epoch % 100 == 0:
    print("Epoch: %d, loss: %1.5f" % (epoch, loss.item())) 

Epoch: 0, loss: 0.33630
Epoch: 100, loss: 0.01541
Epoch: 200, loss: 0.01196
Epoch: 300, loss: 0.00754
Epoch: 400, loss: 0.00617
Epoch: 500, loss: 0.00541
Epoch: 600, loss: 0.00433
Epoch: 700, loss: 0.00350
Epoch: 800, loss: 0.00307
Epoch: 900, loss: 0.00265


In [14]:
df_X_ss = ss.transform(data.iloc[:, 1:]) 
df_X_ss.shape

(150, 74)

In [16]:
df_X_ss = ss.transform(data.iloc[:, 1:]) #old transformers
df_y_mm = mm.transform(data.iloc[:, -1:]) #old transformers

df_X_ss = Variable(torch.Tensor(df_X_ss)) #converting to Tensors
df_y_mm = Variable(torch.Tensor(df_y_mm))

#reshaping the dataset
df_X_ss = torch.reshape(df_X_ss, (df_X_ss.shape[0], 1, df_X_ss.shape[1]))

C:\Users\keato\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- year
Feature names seen at fit time, yet now missing:
- n

  warnings.warn(message, FutureWarning)


In [ ]:
df_X_ss = ss.transform(data.iloc[:, 1:]) #old transformers
df_y_mm = mm.transform(data.iloc[:, -1:]) #old transformers
df_X_ss

In [ ]:
import matplotlib.pyplot as plt
train_predict = lstm1(df_X_ss)#forward pass
data_predict = train_predict.data.numpy() #numpy conversion
dataY_plot = df_y_mm.data.numpy()

data_predict = mm.inverse_transform(data_predict) #reverse transformation
dataY_plot = mm.inverse_transform(dataY_plot)
plt.figure(figsize=(10,6)) #plotting
plt.axvline(x=135, c='r', linestyle='--') #size of the training set

plt.plot(dataY_plot, label='Actual Data') #actual plot
plt.plot(data_predict, label='Predicted Data') #predicted plot
plt.title('Time-Series Prediction')
plt.legend()
plt.show()

In [ ]:
train_pred = lstm1(X_train_tensors)#forward pass
train_pred = train_pred.data.numpy() #numpy conversion
plot_eval(y_train, train_pred)